This is the inverted index part of this project.

Some current thoughts
1. read in the json file
2. pre-process the ingredients
   1. this might be a sample one for now, we'll need Jess's clean data. 
3. build the inverted index for ingredient
4. save the inverted index

In [1]:
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import string

class InvertedIndex:
    def __init__(self):
        self.DFpostings = {}

    def preProcess(self, text):
        #define stemmer and stopwords
        stemmer = SnowballStemmer("english")
        stop_words = set(stopwords.words('english'))

        #clean and tokenize
        text = text.lower()
        text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])
        tokens = word_tokenize(text)
        filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words and len(word) > 1]

        return filtered_tokens

    def indexFile(self, ingredients, fileId):
        processed_ingredients = self.preProcess(' '.join(ingredients))
        for token in set(processed_ingredients):
            if token in self.DFpostings:
                self.DFpostings[token].append(fileId)
            else:
                self.DFpostings[token] = [fileId]

    def save(self):
        with open("DFPostings.json", "w") as file:
            json.dump(self.DFpostings, file)

#load json data
with open('./sample_data.json', 'r') as file:
    recipes = json.load(file)

#initialize inverted index
index = InvertedIndex()

#index each recipe's ingredients
for recipe_id, recipe_info in recipes.items():
    index.indexFile(recipe_info['ingredients'], recipe_id)

#save the index to a file
index.save()
